# Problem Statement

## Context
BoomBikes, a US bike-sharing provider, aims to predict the demand for shared bikes post-Covid to boost revenue post-pandemic.
They have gathered a dataset on daily bike demands in the American market to understand the factors influencing bike demand.

## Goal
Model the bike demand using independent variables to adjust business strategies and meet customer expectations.

# Approach

Check if linear regression can be applied to model the `cnt` variable

**Approach for linear regression** -
1. **Data Preparation**
   - Collect and clean data: Handle missing values, duplicates, and outliers.
2. **Explore the data**
   - Visualize relationships between predictors and target variables using scatter plots, pair plots or correlation matrices
   - Perform Univariate, Bivariate and Multivariate analysis
3. **Build the model**
   - Feature selection: Choose relevant predictors. Transform features if necessary (e.g., scaling).
   - Split data: Divide the dataset into training and testing sets (e.g., 70%-30%).
   - Check assumptions:
     - Linearity between predictors and the target.
     - No multicollinearity among predictors.
   - Fit the model
   - Check coefficients
4. **Evaluate the model**
   - Performance metrics: Use metrics like:
       - R²: Measures variance explained by the model.
       - Adjusted R²: Adjusts R² for the number of predictors.
       - Mean Squared Error (MSE) or Root Mean Squared Error (RMSE).
   - Residual analysis:
       - Check residual plots for patterns.
       - Ensure residuals have constant variance and are normally distributed.
5. **Refine the Model**
   - Remove insignificant predictors: Simplify the model by dropping non-contributory features.
   - Check for overfitting: Ensure the model generalizes well to the test data.

# Solution

## Import Libraries

In [ ]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

In [ ]:
import statsmodels
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Load Data

In [ ]:
df = pd.read_csv('day.csv')
df.head()

## Data Preparation

**Actions:**

1. `casual` and `registered` have a very strong positive correlation with cnt since cnt is their sum. Hence these should be removed.
2. Remove `dteday` because it does not help the cause
3. Map `yr` values to `0: 2018` and 1: 2019`
4. Convert `weathersit` and `season` into categorical variables

`weathersit`:
1: Clear
2: Mist
3: LightRain
4: HeavyRain

`season`:
1:spring, 2:summer, 3:fall, 4:winter

### Check for missing values

In [ ]:
print(df.isnull().sum())

### Map `yr` values to `0: 2018` and `1: 2019`

In [ ]:
df.yr.value_counts()

In [ ]:
mapping_yr = {0: 2018, 1: 2019}
df.yr = df.yr.map(mapping_yr)

In [ ]:
df.yr.value_counts()

### Convert `weathersit` and `season` into categorical variables

In [ ]:
print(df.weathersit.value_counts())
print(df.season.value_counts())

In [ ]:
mapping_weathersit = {1: 'Clear', 2: 'Mist', 3: 'LightRain', 4: 'HeavyRain'}
mapping_season = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}

df.weathersit = df.weathersit.map(mapping_weathersit)
df.season = df.season.map(mapping_season)

In [ ]:
df.weathersit.value_counts()

In [ ]:
df.season.value_counts()

### Remove `instant`, `dteday`, `casual` and `registered` because they do not help the cause

`casual + registered = cnt` Hence there's a high collinearity between casual, registered and cnt. So, casual & registered can be removed. 

In [ ]:
df = df.drop(['dteday', 'casual', 'registered', 'instant'], axis=1)

In [ ]:
df.head()

## Exploratory Data Analysis

### Univariate Analysis

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(15, 15))
plt.subplot(3, 3, 1)
sns.histplot(df.cnt, kde=True)
plt.title('Distribution of Total Bike Rentals (cnt)')

plt.subplot(3, 3, 2)
sns.histplot(df.mnth, kde=True, color='salmon', edgecolor='black')
plt.title('Distribution of Month')

plt.subplot(3, 3, 3)
sns.histplot(df.weathersit, kde=True, color='lightgreen', edgecolor='black')
plt.title('Distribution of Weather')

plt.subplot(3, 3, 4)
sns.histplot(df.season, kde=True, color="skyblue", edgecolor="black")
plt.title('Distribution of season')

plt.subplot(3, 3, 5)
sns.boxplot(df.temp, color="orange")
plt.title('Distribution of Temperature')

plt.subplot(3, 3, 6)
sns.boxplot(df.hum, color="magenta")
plt.title('Distribution of Humidity')

plt.subplot(3, 3, 7)
sns.boxplot(df.windspeed, color="aqua")
plt.title('Distribution of Windspeed')

plt.tight_layout()
plt.show()

**Insights**:
> 1. The distribution of cnt is slightly right skewed, it indicates that a few days have significantly higher rentals compared to others
> 2. The most frequently occurring weather is clear and mist
> 3. Median temp is 20 while that for the humidity is 60 and for windspeed is 12. Such weather conditions seem to be favourable for the season.

### Bivariate analysis

In [ ]:
sns.pairplot(data=df)

**Insights**
> From this analysis, it seems that
> 1. The target variable `cnt` has a good linear relationship with `temp` and `atemp`.
> 2. The count of bike sharings has significantly increased in the year 2019.
> 3. The number of bike sharings is higher in the months from April to October.
> 4. The factors humidity (`hum`) and `windspeed` do not have linear correlation with the `cnt`

#### Visualizing Categorical Variables

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(3,3,1)
sns.boxplot(x = 'yr', y = 'cnt', data = df)
plt.subplot(3,3,2)
sns.boxplot(x = 'mnth', y = 'cnt', data = df)
plt.subplot(3,3,3)
sns.boxplot(x = 'holiday', y = 'cnt', data = df)
plt.subplot(3,3,4)
sns.boxplot(x = 'weekday', y = 'cnt', data = df)
plt.subplot(3,3,5)
sns.boxplot(x = 'workingday', y = 'cnt', data = df)
plt.subplot(3,3,6)
sns.boxplot(x = 'season', y = 'cnt', data = df)
plt.subplot(3,3,7)
sns.boxplot(x = 'weathersit', y = 'cnt', data = df)
plt.show()


**Insights**
> From the above charts, `cnt` increases year over year, it is highest in the months of April to October, highest on non holidays, and high in Summer and fall, high in non rainy days.

### Multivariate Analysis

In [ ]:
plt.figure(figsize = (16, 10))
sns.heatmap(df[['yr', 'mnth', 'holiday', 'weekday', 'workingday', 'temp', 'atemp',	'hum',	'windspeed', 'cnt']].corr(), annot = True)
plt.show()

**Insights**
> From the above heatmap, `cnt` has a high correlation with `yr`, `temp` & `atemp`

### Insights from EDA

1. Distribution of Total Bike Rentals (cnt):
The distribution is slightly right-skewed, indicating a higher frequency of lower rental counts.
3. Bike Rentals by Season:
Rentals vary significantly across seasons:
Fall seems to have the highest median rental count.
Spring has the lowest median rental count.
4. Bike Rentals by Weather Situation:
Rentals decrease as weather worsens:
Clear or Partly Cloudy weather corresponds to the highest rental counts.
Heavy Rain or Snow shows the lowest rental counts.

## Build the model

### Dummy Variables

We'll have to transform the categories of the categorical variables into dummy variables and concat them in the original dataset.

In [ ]:
df.info(verbose=True)

In [ ]:
season = pd.get_dummies(df['season'], dtype='int', drop_first=True)
weathersit = pd.get_dummies(df['weathersit'], dtype='int', drop_first=True)

In [ ]:
season

In [ ]:
df.weathersit.value_counts()

In [ ]:
df = pd.concat([df, season, weathersit], axis = 1)

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['season', 'weathersit'])
df.head()

### Feature Slection

The target variable is `cnt`. Let's choose all the other variables as predictors to start with and remove predictors as and when required.

### Split the data

In [ ]:
y = df.cnt

In [ ]:
df.shape

In [ ]:
X = df

In [ ]:
X.pop('cnt')

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Scaling

In [ ]:
X_train.head()

In [ ]:
scaler = MinMaxScaler()
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables

num_vars = ['yr', 'mnth', 'holiday', 'weekday', 'workingday','temp', 'atemp', 'hum', 'windspeed', 'Spring', 'Summer', 'Winter', 'LightRain', 'Mist']

X_train[num_vars] = scaler.fit_transform(X_train[num_vars])

In [ ]:
X_train.head()

### Check Multicolinearity

Variance Inflation Factor or VIF, gives a basic quantitative idea about how much the feature variables are correlated with each other. It is an extremely important parameter to test our linear model.

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
def calculate_vif(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif

calculate_vif(X_train)

The VIF is too high for columns `temp` and `atemp`.

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated

plt.figure(figsize = (16, 10))
sns.heatmap(X_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

`temp` and `atemp` are highly correlated, so, we should keep only one of two.
There's a high correlation between `winter` and `month` & `temp` and `fall`. But we'll deal with them later.

In [ ]:
X_train = X_train.drop(columns=['atemp'])
calculate_vif(X_train)

We still have high VIF for mnth So, let's drop mnth.

In [ ]:
X_train = X_train.drop(columns=['mnth'])
calculate_vif(X_train)

### Train the model

In [ ]:
def buildLinearRegModel(X, Y):
    X_sm = sm.add_constant(X)
    lr = sm.OLS(Y, X_sm).fit()
    return lr

In [ ]:
lr1 = buildLinearRegModel(X_train, y_train)
print(lr1.summary())

Notice that all the p values are well below the limit of 0.05

In [ ]:
# Plotting the coefficients

coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr1.params[1:]
}).sort_values(by='Coefficient', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x='Coefficient', y='Feature', data=coefficients)
plt.title('Feature Importance')
plt.show()

## Evaluate the model

1. Residual analysis of the train data
3. Use X_test to predict the `cnt`.
4. Check R squared value

### Residual analysis

So, now to check if the error terms are also normally distributed (which is infact, one of the major assumptions of linear regression), let us plot the histogram of the error terms and see what it looks like.

In [ ]:
X_train_sm = sm.add_constant(X_train)
y_train_pred = lr1.predict(X_train_sm)

In [ ]:
# Calculate residuals
residuals = y_train - y_train_pred

# Compute the mean of the residuals
residual_mean = np.mean(residuals)
print(f'Mean of Residuals: {residual_mean}')

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.histplot((y_train - y_train_pred), bins = 20, kde=True)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

In [ ]:
# Let's now plot the graph for actual versus predicted values.

fig = plt.figure()
plt.scatter(y_train, y_train_pred)
fig.suptitle('y_train vs y_train_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_train', fontsize = 18)                          # X-label
plt.ylabel('y_train_pred', fontsize = 16)      

Let's plot a graph to check Homoscedasticity (Constant variance of error terms)

In [ ]:
# Residuals vs. Predicted Plot (Homoscedasticity)
plt.scatter(lr1.fittedvalues, lr1.resid)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

**Observations:**  

The residuals scatter randomly and evenly around the horizontal axis (y = 0). There is no clear pattern or systematic change in the spread of residuals. This indicates that the variance of residuals is constant across all levels of predicted values.

### Making Predictions on the test data using the model

Use X_test to predict y_test using the model lr

In [ ]:
print(X_test.shape)
print(X_train.shape)

Drop unnecessary columns from X_test

In [ ]:
X_test.columns

In [ ]:
X_train.columns

In [ ]:

num_vars = ['yr', 'mnth', 'holiday', 'weekday', 'workingday', 'temp', 'atemp',
       'hum', 'windspeed', 'Spring', 'Summer', 'Winter', 'LightRain', 'Mist']
X_test[num_vars] = scaler.transform(X_test[num_vars])

In [ ]:
X_test.drop(columns=['atemp', 'mnth'], axis=1, inplace=True)

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
# Making predictions using the lr model
X_test_sm = sm.add_constant(X_test)
y_test_pred = lr1.predict(X_test_sm)

In [ ]:
# Calculate residuals
residuals = y_test - y_test_pred

# Compute the mean of the residuals
residual_mean = np.mean(residuals)
print(f'Mean of Residuals: {residual_mean}')

In [ ]:
# Let's now plot the graph for actual versus predicted values.

fig = plt.figure()
plt.scatter(y_test, y_test_pred)
fig.suptitle('y_test vs y_pred', fontsize = 20)              # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_pred', fontsize = 16)      

In [ ]:
# Check the r2 score
print('R squared of the test data using the linear regression model is: ', r2_score(y_test, y_test_pred))

## Conclusion

We built a linear regression model `lr1` to predict the bike sharing count. This model produced an R squared value of 0.809 for the test data set.

In [ ]:
print(lr1.summary())

We can see that the equation of our best fitted line is:  

`cnt = 2015.9015 * yr - 557.7596 * holiday + 422.7937 * weekday + 162.5193 * workingday + 4288.2546 * temp - 1073.4476 * hum - 1564.5809 * windspeed - 644.9492 * Spring + 377.0228 * Summer + 786.0812 * Winter - 2167.2031 * LightRain - 509.4176 * Mist + 2159.4744`

**R-squared of the model: 0.834**  
**R-squared of the test data: 0.809**